This notebook demonstrates LD (linkage disequilibrium) calculations.

Let's first load some example data that will allow us to calculate LD (r^2 and D') for two specific SNPs.
The input file is in the "Linkage format" of Haploview 4.2, as described [here](
http://www.broadinstitute.org/science/programs/medical-and-population-genetics/haploview/input-file-formats-0).

In [265]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('just2.chr-6.ped', sep='\t', header=None, 
                 names=['FamId', 'Id', 'FatherId', 'MotherId', 'Sex', 'Affection', 'A1', 'A2', 'B1', 'B2'],
                 index_col=1)
df = df[['A1', 'A2', 'B1', 'B2']]  # prune all irrelevant columns
df[:6]

,A1,A2,B1,B2
Id,,,,
id1,A,G,G,A
id2,A,G,G,A
id3,A,A,A,A
id4,A,A,G,A
id5,A,A,A,A
id6,G,G,G,G


The table above shows subset of 6 individuals, and their genotype at two locus - A and B. Each locus is represented at two homologous chromosomes, hence 4 columns in the table.

Let's now calculate 3x3 square of all frequencies:

In [282]:
df['A'] = df['A1'] + df['A2']
df['B'] = df['B2'] + df['B1']
df['count'] = 1
freq = df[['count', 'A', 'B']].groupby(['A', 'B']).sum().unstack(level=1)['count']
freq.fillna(0, inplace=True)
freq

B,AA,AG,GG
A,,,
AA,83.0,2.0,0.0
AG,50.0,133.0,0.0
GG,7.0,45.0,59.0


The next step is to perform [Haplotype_resolution](https://en.wikipedia.org/wiki/Haplotype#Haplotype_resolution), as described in the following papers:
* [W. G. Hill - Estimation of linkage disequilibrium in randomly mating populations](http://www.nature.com/hdy/journal/v33/n2/abs/hdy197489a.html)
* [Gaunt at all - Cubic exact solutions for the estimation of pairwise haplotype frequencies](http://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-8-428)

TBD: make this work, because at the moment results don't match with plink, which gives the following haplotype frequencies:
```
E:\sandbox>plink --bfile chr6 --ld rs1928160 rs62404665
PLINK v1.90b3.38 64-bit (7 Jun 2016)       https://www.cog-genomics.org/plink2
(C) 2005-2016 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to plink.log.
Options in effect:
  --bfile chr6
  --ld rs1928160 rs62404665

32693 MB RAM detected; reserving 16346 MB for main workspace.
1925995 variants loaded from .bim file.
379 people (379 males, 0 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 379 founders and 0 nonfounders present.
Calculating allele frequencies... done.
1925995 variants and 379 people pass filters and QC.
Note: No phenotypes present.

--ld rs1928160 rs62404665:

   R-sq = 0.545885       D' = 0.983245

   Haplotype     Frequency    Expectation under LE
   ---------     ---------    --------------------
          AG      0.003068                0.183085
          GG      0.390072                0.210055
          AA      0.462632                0.282614
          GA      0.144229                0.324246

   In phase alleles are AA/GG
```

In [333]:
AA = 'AA'; AG = 'AG'; GG = 'GG';

# Incomplete distributions of haplotype frequencies
# The question is where to put freq[AG][AG], which can contribute to all haplotypes
X_AA = 2 * freq[AA][AA] + freq[AA][AG] + freq[AG][AA]
X_GG = 2 * freq[GG][GG] + freq[AG][GG] + freq[GG][AG]
X_AG = 2 * freq[AA][GG] + freq[AG][GG] + freq[AA][AG]
X_GA = 2 * freq[GG][AA] + freq[GG][AG] + freq[AG][AA]

N = float(X_AA + X_GG + X_AG + X_GA + 2 * freq[AG][AG])

# Find allele frequencies
p_A = (X_AA + X_AG + freq[AG][AG]) / N
p_B = (X_GG + X_GA + freq[AG][AG]) / N

# Initial approximation for
f_AA = 1 / (4 * N) * (X_AA - X_AG - X_GA + X_GG) + 0.5 - (1 - p_A)*(1 - p_B)
f_AG = p_A - f_AA
f_GA = p_B - f_AA
f_GG = 1 - p_A - p_B + f_AA

for i in xrange(1, 10):
    rAAGG = (f_AA * f_GG) / (f_AA * f_GG + f_AG * f_GA)
    rAGGA = 1 - rAAGG

    new_f_AA = (X_AA + rAAGG * freq[AG][AG]) / (2 * N)
    new_f_GA = (X_GA + rAGGA * freq[AG][AG]) / (2 * N)
    new_f_AG = (X_AG + rAGGA * freq[AG][AG]) / (2 * N)
    new_f_GG = (X_GG + rAAGG * freq[AG][AG]) / (2 * N)

    print f_AA, f_GA, f_AG, f_GG, rAAGG
    
    f_AA = new_f_AA
    f_AG = new_f_AG
    f_GA = new_f_GA
    f_GG = new_f_GG
    



0.35046922536 0.0426706163282 0.256390932951 0.35046922536 0.918214563169
0.224355235423 0.0084943687985 0.0790748437325 0.188075552046 0.984330972171
0.230155685553 0.00269391866839 0.0732743936024 0.193876002176 0.995595729601
0.231143952531 0.00170565169067 0.0722861266247 0.194864269154 0.997270121222
0.231290848366 0.00155875585587 0.0721392307899 0.195011164988 0.997513145897
0.231312169132 0.00153743508953 0.0721179100236 0.195032485755 0.997548293863
0.231315252694 0.00153435152789 0.0721148264619 0.195035569316 0.99755337459
0.23131569843 0.00153390579129 0.0721143807253 0.195036015053 0.997554108967
0.231315762858 0.00153384136375 0.0721143162978 0.195036079481 0.997554215114


Some references:
* Haploview program is taken from
  [here](https://www.broadinstitute.org/scientific-community/science/programs/medical-and-population-genetics/haploview/haploview)

* plink2 can be downloaded from https://www.cog-genomics.org/plink2/

The input data was prepared with the following commands

```
plink --bfile chr6 --recode HV --snps-only no-DI
      --from-kb 22000 --to-kb 22500 --chr 6 --allele1234 --maf  --bp-space 1000
      
plink --bfile chr6 --recode HV --snps-only no-DI --snps rs1928160,rs62404665 --chr 6  --out just2
```

Estimation of haplotype frequencies, online page:

* http://www.genes.org.uk/cgi-bin/cubex.py